# Term Identifier Analysis

This notebook looks for local unique identifiers that are the same in different biomedical ontologies, databases, and identifier spaces. The fact that there are lots of overlaps shows that the need for well-defined prefixes is incredibly important.

In [17]:
import pyobo
import bioregistry
from tqdm.auto import tqdm
from tqdm.contrib.logging import logging_redirect_tqdm
from itertools import combinations
import pandas as pd
import json
from operator import itemgetter
import networkx as nx

In [2]:
def print_overlap(a: set[str], b: set[str]):
    intersection = set(a).intersection(b)
    union = set(a).union(b)
    print(
        f"{len(intersection):,} same, {len(union):,} total, "
        f"{len(intersection)/min(len(a), len(b)):.1%} overlap"
    )

Extract the list of local unique identifiers from all resources gettable from PyOBO. Note that these local unique identifiers are normalized by the Bioregistry to remove potential redundant prefixes.

In [3]:
ids = {}
resources = [
    resource 
    for resource in bioregistry.resources() 
    if resource.has_download() and not resource.is_deprecated()
]
with logging_redirect_tqdm():
    it = tqdm(resources, unit="prefix", unit_scale=True)
    failed_prefixes = set()
    succeeded_prefixes = set()
    for resource in it:
        it.set_postfix(failed=len(failed_prefixes), succeeded=len(succeeded_prefixes))
        try:
            resource_ids = pyobo.get_ids(resource.prefix, strict=False)
        except Exception:
            tqdm.write(f"failed on {resource.prefix}")
            failed_prefixes.add(resource.prefix)
            continue
        else:
            if resource_ids:
                ids[resource.prefix] = resource_ids
            succeeded_prefixes.add(resource.prefix)

  0%|          | 0.00/332 [00:00<?, ?prefix/s]

failed on ado


INFO: [2023-02-10 11:45:41] pystow.utils - downloading with urllib from http://purl.allotrope.org/voc/afo/merged-OLS/REC/2019/05/10 to /Users/cthoyt/.data/pyobo/raw/afo/2019-05-10/10


failed on aeon
failed on afo
failed on apaonto
failed on apollosv


[Fatal Error] :67773:173: The reference to entity "idAhol" must end with the ';' delimiter.


...

INFO: [2023-02-10 12:15:55] pystow.utils - downloading with urllib from https://raw.githubusercontent.com/cthoyt/vsmo/main/vsmo.obo to /Users/cthoyt/.data/pyobo/raw/vsmo/vsmo.obo


failed on vo


INFO: [2023-02-10 12:15:56] pyobo.reader - [vsmo] parsing with obonet from /Users/cthoyt/.data/pyobo/raw/vsmo/vsmo.obo
INFO: [2023-02-10 12:15:57] pyobo.reader - [vsmo] extracting OBO using obonet
INFO: [2023-02-10 12:15:57] pyobo.reader - [vsmo] does not report a name
INFO: [2023-02-10 12:15:57] pyobo.reader - [vsmo] using version 0.1.0
INFO: [2023-02-10 12:16:01] pyobo.reader - [vsmo] got 2,243 references, 30 typedefs, 1,902 terms, 0 alt ids, 1,906 parents, 1,480 synonyms, 1,134 xrefs, 1,996 relations, and 0 properties


failed on vso


INFO: [2023-02-10 12:16:17] pyobo.getters - [wikipathways] caching nomenclature plugin
INFO: [2023-02-10 12:16:17] pystow.utils - downloading with urllib from http://data.wikipathways.org/20230210/gmt/wikipathways-20230210-gmt-Anopheles_gambiae.gmt to /Users/cthoyt/.data/pyobo/raw/wikipathways/20230210/wikipathways-20230210-gmt-Anopheles_gambiae.gmt
INFO: [2023-02-10 12:16:18] pystow.utils - downloading with urllib from http://data.wikipathways.org/20230210/gmt/wikipathways-20230210-gmt-Arabidopsis_thaliana.gmt to /Users/cthoyt/.data/pyobo/raw/wikipathways/20230210/wikipathways-20230210-gmt-Arabidopsis_thaliana.gmt
INFO: [2023-02-10 12:16:18] pystow.utils - downloading with urllib from http://data.wikipathways.org/20230210/gmt/wikipathways-20230210-gmt-Bos_taurus.gmt to /Users/cthoyt/.data/pyobo/raw/wikipathways/20230210/wikipathways-20230210-gmt-Bos_taurus.gmt
INFO: [2023-02-10 12:16:19] pystow.utils - downloading with urllib from http://data.wikipathways.org/20230210/gmt/wikipathways

In [25]:
rows = []
prefixes_encountered = set()
total = len(ids) * (len(ids) - 1) / 2
min_terms = 10
for (a_prefix, a), (b_prefix, b) in tqdm(combinations(ids.items(), 2), total=total, unit_scale=True):
    len_a, len_b = len(a), len(b)
    if len_a < min_terms or len_b < min_terms:
        continue

    prefixes_encountered.add(a_prefix)
    prefixes_encountered.add(b_prefix)
    
    intersection = set(a).intersection(b)
    if not intersection:
        continue
    
    n_intersection = len(intersection)
    overlap = n_intersection / min(len_a, len_b)
    rows.append((a_prefix, b_prefix, len_a, len_b, n_intersection, round(overlap, 5)))  
    
# Sort descending by normalized overlap
rows = sorted(rows, key=itemgetter(5), reverse=True)
overlap_df = pd.DataFrame(
    rows, 
    columns=[
        "source_prefix", "target_prefix", "source_size",
        "target_size", "absolute_overlap", "normalized_overlap",
    ],
)
overlap_df.to_csv("edges.tsv", sep='\t', index=False)
overlap_df

  0%|          | 0.00/16.7k [00:00<?, ?it/s]

,source_prefix,target_prefix,source_size,target_size,absolute_overlap,normalized_overlap
0,apo,lbo,309,1108,309,1.00000
1,apo,sbo,309,694,309,1.00000
2,bfo,bto,35,6511,35,1.00000
3,bfo,chmo,35,2908,35,1.00000
4,bfo,cmo,35,3025,35,1.00000
...,...,...,...,...,...,...
5220,doid,efo,11207,15540,1,0.00009
5221,doid,mp,11207,13569,1,0.00009
5222,fbbt,tto,17570,38640,1,0.00006
5223,fbbt,xpo,17570,20061,1,0.00006


In [26]:
prefix_rows = [
    (
        prefix,
        bioregistry.get_name(prefix),
        bioregistry.get_example(prefix),
        len(ids[prefix]),
    )
    for prefix in sorted(prefixes_encountered)
]
prefix_df = pd.DataFrame(
    prefix_rows,
    columns=["prefix", "name", "example_luid", "n_luids"]
)
prefix_df.to_csv("nodes.tsv", sep='\t', index=False)
prefix_df

,prefix,name,example_luid,n_luids
0,agro,Agronomy Ontology,00020007,772
1,aism,Ontology for the Anatomy of the Insect Skeleto...,0000027,553
2,amphx,The Amphioxus Development and Anatomy Ontology,1000160,399
3,apo,Ascomycete phenotype ontology,0000184,309
4,aro,Antibiotic Resistance Ontology,1000001,6610
...,...,...,...,...
128,xpo,Xenopus Phenotype Ontology,0100002,20061
129,zeco,Zebrafish Experimental Conditions Ontology,0000171,161
130,zfa,Zebrafish anatomy and development ontology,0005926,3079
131,zfs,Zebrafish developmental stages ontology,0000050,54


In [27]:
graph = nx.Graph()
for prefix, name, _example, size in prefix_rows:
    graph.add_node(
        prefix, name=name, size=size, prefix=prefix, 
        description=bioregistry.get_description(prefix),
    )
for source, target, _, _, absolute_overlap, normalized_overlap in rows:
    graph.add_edge(
        source, target, weight=normalized_overlap, absolute_overlap=absolute_overlap,
    )

In [28]:
nx.write_graphml_lxml(graph, "graph.graphml")

In [29]:
with open("graph.cytoscape.json", "w") as file:
    json.dump(nx.cytoscape_data(graph), file, indent=2)